In [0]:
import dlt
from pyspark.sql import functions as F

The Delta Live Tables (DLT) module is not supported on this cluster.
 You should either create a new pipeline or use an existing pipeline to run DLT code.

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-8313695966771713>, line 1
----> 1 import dlt
      2 from pyspark.sql import functions as F

ModuleNotFoundError: No module named 'dlt'

Data Bricks Free Version Not Supporting DLT, So proceading with normal method.

In [0]:
@dlt.table(
    name="bronze_customers",
    comment="Raw customers data from source"
)
def bronze_customers():
    return (
        spark.readStream
        .format("delta")
        .table("my_catalog.bronze_retailx.customers")
    )


In [0]:
@dlt.view
def customers_clean():
    return (
        dlt.read("bronze_customers")
        .withColumn("customer_id", F.col("CUSTOMER_ID").cast("long"))
        .withColumn("created_date", F.to_date("CREATED_DATE"))
        .withColumn("email", F.lower(F.trim("EMAIL")))
        .withColumn("name", F.initcap(F.trim("NAME")))
        .filter(F.col("customer_id").isNotNull())
)

In [0]:
dlt.apply_changes(
    target = "silver_customers",
    source = "customers_clean",
    keys = ["customer_id"],
    sequence_by = F.col("_ingested_at"),
    stored_as_scd_type = 2,
    apply_as_deletes = None,
    except_column_list = [
        "_ingested_at",
        "_source_file",
    ]
)